# Q&A Chat bot using Ollama (Complete offline)

So, here I am trying to create a RAG Q&A Bot which will be able to get context data from a web URL and then use that information as context to answer any question.

This notebook will try to first load the data from a web URL using the Langchain Document loaders -> WebBasedLoader

Right now, I am loading my blog article on Franken PHP and trying to ask questions on that blog post.

In [1]:
from langchain_community.document_loaders import WebBaseLoader

In [2]:
url = "https://www.amitavroy.com/articles/beyond-boundaries-how-frankenphp-redefines-php-application-runtimes-2024-01-01"

This is where we loaded the URL using WebBaseLoader and then got the entire document in the "docs" variable using the loader.load method.

In [3]:
loader = WebBaseLoader(url)
docs = loader.load()

## Embeddings

In [5]:
from langchain_community.embeddings import OllamaEmbeddings
embeddings = OllamaEmbeddings()

## Getting the FAISS vector store

In [7]:
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [8]:
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

## Getting the LLM

In [9]:
from langchain_community.llms import Ollama
llm = Ollama(model="llama2")

## Getting the Prompt set up

In [11]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")
document_chain = create_stuff_documents_chain(llm, prompt)

In [12]:
from langchain.chains import create_retrieval_chain
retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [13]:
question = "How can FrankenPHP speed things up?"
response = retrieval_chain.invoke({"input": question})
print(response["answer"])

According to the context, FrankenPHP can speed things up in several ways:

1. Early hint support: FrankenPHP claims a whopping 30% reduction in loading time by harnessing early hint support, as mentioned in the context.
2. Worker Script Mode: By using worker mode, FrankenPHP can power up performance by running entirely from memory, providing a significant boost for high-traffic scenarios.
3. Prometheus metrics and tracing: With built-in support for Prometheus metrics and tracing, FrankenPHP can help keep an eye on things and monitor the application's performance in real-time.


In [14]:
question = "Can I run a Laravel binary and run it?"
response = retrieval_chain.invoke({"input": question})
print(response["answer"])

Based on the provided context, the answer to your question is yes, you can run a Laravel binary using FrankenPHP. The context mentions that FrankenPHP supports running PHP applications as a standalone binary, which implies that it can handle various PHP versions and frameworks, including Laravel.

FrankenPHP is designed to be a modern PHP app server that offers numerous advantages over traditional PHP approaches. It uses Go's goroutine features and relies on Caddy server for HTTPS support in local development. FrankenPHP also supports HTTP/1.1, HTTP/2, and HTTP/3 protocols, which can help reduce round-trip times and improve performance.

Since Laravel is a PHP framework, it should be compatible with FrankenPHP's standalone binary approach. Therefore, you can create a Laravel binary using the FrankenPHP build tool and run it as a standalone application. This could simplify your deployment process and reduce the attack surface area by using fewer software packages in production.

However